# Sample Usage

In [1]:
from utils import convert_to_json
from metric.evaluator import get_evaluator

task = 'dialogue'

# a list of dialogue histories
src_list = ['hi , do you know much about the internet ? \n i know a lot about different sites and some website design , how about you ? \n\n']
# a list of additional context that should be included into the generated response
context_list = ['the 3 horizontal line menu on apps and websites is called a hamburger button .\n']
# a list of model outputs to be evaluated
output_list = ['i do too . did you know the 3 horizontal line menu on apps and websites is called the hamburger button ?']

# Prepare data for pre-trained evaluators
data = convert_to_json(output_list=output_list, 
                       src_list=src_list, context_list=context_list)
# Initialize evaluator for a specific task
evaluator = get_evaluator(task)
# Get multi-dimensional evaluation scores
eval_scores = evaluator.evaluate(data, print_result=True)

Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.



Evaluation scores are shown below:
+-------------------+----------+
|     Dimensions    |  Score   |
+-------------------+----------+
|    naturalness    | 0.950217 |
|     coherence     | 0.973135 |
|    engagingness   | 1.750486 |
|    groundedness   | 0.999566 |
| understandability | 0.946209 |
|      overall      | 1.123923 |
+-------------------+----------+


In [2]:
from utils import convert_to_json
from metric.evaluator import get_evaluator

task = 'dialogue'

# Batch inputs: multiple dialogue histories, contexts, and model outputs
src_list = [
    'hi , do you know much about the internet ? \n i know a lot about different sites and some website design , how about you ? \n\n',
    'what is your favorite color ? \n i like blue a lot , but sometimes i prefer green . \n\n'
]
context_list = [
    'the 3 horizontal line menu on apps and websites is called a hamburger button .\n',
    'colors can reflect your mood and personality .\n'
]
output_list = [
    'i do too . did you know the 3 horizontal line menu on apps and websites is called the hamburger button ?',
    'i like blue as well . it is calming and reminds me of the ocean .'
]

# Prepare data for pre-trained evaluators
data = convert_to_json(output_list=output_list, 
                       src_list=src_list, context_list=context_list)

# Initialize evaluator for a specific task
evaluator = get_evaluator(task)

# Get multi-dimensional evaluation scores for the batch
eval_scores = evaluator.evaluate(data, print_result=False)

# Display scores
for i, score in enumerate(eval_scores):
    print(f"Scores for example {i + 1}: {score}")


Scores for example 1: {'naturalness': 0.9502174201360719, 'coherence': 0.9731347836152868, 'engagingness': 1.7504860805525295, 'groundedness': 0.9995656267195939, 'understandability': 0.9462095037239142, 'overall': 1.1239226829494793}
Scores for example 2: {'naturalness': 0.9675946477090701, 'coherence': 0.998674558536015, 'engagingness': 1.9936029005678884, 'groundedness': 0.9857853472625128, 'understandability': 0.9632170362278318, 'overall': 1.1817748980606635}


In [3]:
eval_scores

[{'naturalness': 0.9502174201360719,
  'coherence': 0.9731347836152868,
  'engagingness': 1.7504860805525295,
  'groundedness': 0.9995656267195939,
  'understandability': 0.9462095037239142,
  'overall': 1.1239226829494793},
 {'naturalness': 0.9675946477090701,
  'coherence': 0.998674558536015,
  'engagingness': 1.9936029005678884,
  'groundedness': 0.9857853472625128,
  'understandability': 0.9632170362278318,
  'overall': 1.1817748980606635}]

# Response Generation Evaluation

In [4]:
import pandas as pd
import warnings
import logging

warnings.filterwarnings("ignore")
logging.getLogger('transformers').setLevel(logging.ERROR)

# Set the logging level to ERROR to ignore warnings
logging.getLogger("transformers").setLevel(logging.ERROR)


In [6]:
Dataset = "FoCus"                                             # Blended Skill Talk, FoCus, IT-ConvAI2
LLM_name = "Qwen2-7B-Instruct"                                # Mistral-7B-Instruct, Llama3-1-8B-Instruct, Qwen2-7B-Instruct,  gpt-3.5-turbo, gpt-4-turbo, gpt-4o-mini
COT_SETUP = True
TOPIC_MODE = "general"          #general, detailed

In [8]:
df = pd.read_csv(f'../prompts/{Dataset}_{TOPIC_MODE}.csv')
print("Shape:", df.shape)

df.head(2)

Shape: (1000, 8)


,personas,context,act_response,OCEAN_type,MBTI_type,contextualized_personas,context_topic,retrieved_persona
0,I would like to visit the Nazareth House again...,User1: I think Ive been there before but I don...,User2: The history of the house you are intere...,Neuroticism,INFP,"{""I would like to visit the Nazareth House aga...",Social,I would like to visit the Nazareth House again.
1,I have been to Vermont a few times to go skiin...,"User1: Wow, this is amazing! What is this?\nUs...",User2: This house was use as a stop for slaves...,Conscientiousness,INFP,"{""I have been to Vermont a few times to go ski...",Social,I have always been fascinated by stories about...


In [9]:
# ### Only For: Blended Skill Talk
if Dataset == "Blended Skill Talk":
    df['personas'] = df['personas'].str.replace(r'\[User 1 persona\]:|\[|\]|"|\'', '', regex=True).str.strip()

# ### Only For: PEC
if Dataset == "PEC":
    df['personas'] = df['personas'].str.replace(r'\[Responder persona\]:|\[|\]|"|\'', '', regex=True).str.strip()


print(df.isnull().sum())
df.head(6)

personas                   0
context                    0
act_response               0
OCEAN_type                 0
MBTI_type                  0
contextualized_personas    0
context_topic              0
retrieved_persona          0
dtype: int64


,personas,context,act_response,OCEAN_type,MBTI_type,contextualized_personas,context_topic,retrieved_persona
0,I would like to visit the Nazareth House again...,User1: I think Ive been there before but I don...,User2: The history of the house you are intere...,Neuroticism,INFP,"{""I would like to visit the Nazareth House aga...",Social,I would like to visit the Nazareth House again.
1,I have been to Vermont a few times to go skiin...,"User1: Wow, this is amazing! What is this?\nUs...",User2: This house was use as a stop for slaves...,Conscientiousness,INFP,"{""I have been to Vermont a few times to go ski...",Social,I have always been fascinated by stories about...
2,I am fascinated by the Spanish Colonial Reviva...,"User1: Wow, this is amazing! What is this?\nUs...","User2: Sure, you will like to know that this p...",Neuroticism,INFP,"{""I am fascinated by the Spanish Colonial Revi...",Social,I have many friends in Marion.
3,I want to become a college student. I want to ...,User1: Where is this place?\nUser2: Hello! Wel...,User2: Technische Universität Darmstadt in the...,Neuroticism,INFP,"{""I want to become a college student."": ""Perso...",Science,I like science.
4,I like to visit england. I love church. I woul...,User1: Where is this place?\nUser2: This place...,"User2: I suggest a place, for your wish of see...",Neuroticism,INFP,"{""I like to visit england."": ""Travel & Destina...",Social,I love church.
5,I would like to go to University. I live in Mi...,User1: I think Ive been there before but I don...,User2: They offer 132 bachelors degree program...,Neuroticism,INFP,"{""I would like to go to University."": ""Work & ...",Social,I live in Michigan.


In [10]:
COT_ = "-COT" if COT_SETUP else ""
 
response = pd.read_csv(f'../Responses/{Dataset}/{LLM_name}{COT_}_{TOPIC_MODE}.csv')
print("Shape:", response.shape)

print("\nMissing Values:")
print(response.isnull().sum())

response

Shape: (1000, 2)

Missing Values:
gen_response     60
response_time     0
dtype: int64


,gen_response,response_time
0,Nazareth House was constructed between 1924 an...,5.057299
1,This historic home served as a stop on the Und...,4.019164
2,The Marion Palace Theatre is an atmospheric st...,4.274230
3,Technische Universität Darmstadt has significa...,5.609988
4,"The Boston Stump, officially known as St Botol...",5.808690
...,...,...
995,"This museum, also known as Gaydons Pharmacy an...",6.194491
996,Mahasthangarh is the site of an ancient city k...,6.432805
997,Armagh County Museum showcases paintings by lo...,4.221210
998,Nyanga National Park offers a diverse range of...,6.020989


In [11]:
# Calculate maximum number of words in each column
max_response_length = response['gen_response'].dropna().apply(lambda x: len(x.split())).max()

print(f"Maximum Response Length (in words): {max_response_length}")

Maximum Response Length (in words): 121


In [12]:
import pandas as pd
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# Initialize stop words
stop_words = set(stopwords.words('english'))

# Function to preprocess text
def preprocess_text(text, remove_stop_words=True):
    if pd.isnull(text):
        return None
    text = text.lower()  # Lowercasing
    text = text.translate(str.maketrans('', '', string.punctuation))  # Removing punctuation
    tokens = word_tokenize(text)  # Tokenization
    if remove_stop_words:
        tokens = [word for word in tokens if word not in stop_words]  # Removing stop words
    return ' '.join(tokens)  # Join tokens back into a single string

# Create eval_df
eval_df = pd.DataFrame({
    'personas': df['personas'],
    'context': df['context'],
    'gen_response': response['gen_response'],
    'response_time': response['response_time']
})

print(eval_df.isnull().sum())
eval_df.head()

personas          0
context           0
gen_response     60
response_time     0
dtype: int64


,personas,context,gen_response,response_time
0,I would like to visit the Nazareth House again...,User1: I think Ive been there before but I don...,Nazareth House was constructed between 1924 an...,5.057299
1,I have been to Vermont a few times to go skiin...,"User1: Wow, this is amazing! What is this?\nUs...",This historic home served as a stop on the Und...,4.019164
2,I am fascinated by the Spanish Colonial Reviva...,"User1: Wow, this is amazing! What is this?\nUs...",The Marion Palace Theatre is an atmospheric st...,4.274230
3,I want to become a college student. I want to ...,User1: Where is this place?\nUser2: Hello! Wel...,Technische Universität Darmstadt has significa...,5.609988
4,I like to visit england. I love church. I woul...,User1: Where is this place?\nUser2: This place...,"The Boston Stump, officially known as St Botol...",5.808690


In [13]:
import torch
device = 0 if torch.cuda.is_available() else -1  # device set to 0 for GPU, -1 for CPU
device

0

**Finalized Input Mapping:**

- src_list: Use the context column (conversation history).
- context_list: Use the flattened and cleaned persona column.
- output_list: Use the gen_response (the response your model generates).

**Note:**

The act_response (true or reference response) is not required as an input for the UniEval evaluation process because UniEval evaluates the generated response (gen_response) based on how well it fits the provided context (conversation history) and additional persona information. 

In [ ]:
from utils import convert_to_json
from metric.evaluator import get_evaluator

def calculate_unieval_scores(personas, contexts, gen_responses):
    """
    Calculates UniEval scores for a batch of inputs.

    Args:
        personas (list): List of persona information as additional context.
        contexts (list): List of conversation histories leading to the responses.
        gen_responses (list): List of generated responses to be evaluated.

    Returns:
        list: A list of dictionaries containing UniEval scores for each input.
    """
    # Flatten personas if they are lists
    personas = [' '.join(p) if isinstance(p, list) else p for p in personas]

    # Prepare inputs for UniEval
    data = convert_to_json(output_list=gen_responses, src_list=contexts, context_list=personas)

    # Initialize the evaluator for dialogue tasks
    evaluator = get_evaluator('dialogue')

    # Evaluate and obtain scores for all inputs
    eval_scores = evaluator.evaluate(data, print_result=False)
    

    return eval_scores


In [15]:
# Define the worst UniEval score as a dictionary
worst_unieval_score = {
    'naturalness': 0.0,
    'coherence': 0.0,
    'engagingness': 0.0,
    'groundedness': 0.0,
    'understandability': 0.0,
    'overall': 0.0
}


In [16]:
from tqdm import tqdm
import pandas as pd

# Function to evaluate in batches or the entire DataFrame
batch_size = 200  # Adjust batch size as needed

# List to store all UniEval scores
all_unieval_scores = []

# Split into batches if necessary
for i in tqdm(range(0, len(eval_df), batch_size), desc="Evaluating batches"):
    batch = eval_df.iloc[i:i+batch_size]

    # Extract relevant fields from the batch
    personas = batch['personas'].tolist()
    contexts = batch['context'].tolist()
    gen_responses = batch['gen_response'].tolist()

    # Check for NaN responses and handle them
    valid_indices = [j for j, response in enumerate(gen_responses) if pd.notna(response) and response.strip() != '']
    invalid_indices = [j for j, response in enumerate(gen_responses) if j not in valid_indices]

    # Prepare valid inputs
    valid_personas = [personas[j] for j in valid_indices]
    valid_contexts = [contexts[j] for j in valid_indices]
    valid_gen_responses = [gen_responses[j] for j in valid_indices]

    # Evaluate valid inputs
    if valid_personas:
        eval_scores = calculate_unieval_scores(valid_personas, valid_contexts, valid_gen_responses)
        all_unieval_scores.extend(eval_scores)

    # Append worst scores for invalid inputs
    all_unieval_scores.extend([worst_unieval_score] * len(invalid_indices))

# Convert all scores into a DataFrame
metrics_df = pd.DataFrame(all_unieval_scores)

# Rename columns for clarity
metrics_df.columns = [
    "UniEval Naturalness",
    "UniEval Coherence",
    "UniEval Engagingness",
    "UniEval Groundedness",
    "UniEval Understandability",
    "UniEval Overall"
]

# Combine with original DataFrame if needed
eval_df = pd.concat([eval_df.reset_index(drop=True), metrics_df.reset_index(drop=True)], axis=1)

metrics_df

Evaluating batches: 100%|██████████| 5/5 [15:59<00:00, 191.84s/it]


,UniEval Naturalness,UniEval Coherence,UniEval Engagingness,UniEval Groundedness,UniEval Understandability,UniEval Overall
0,0.947327,0.997518,1.995575,0.994870,0.942323,1.175523
1,0.922257,0.997019,1.865745,0.937415,0.922912,1.129070
2,0.930815,0.998035,2.723239,0.643350,0.926734,1.244435
3,0.938296,0.998732,2.994051,0.826439,0.937031,1.338910
4,0.728584,0.996909,3.589608,0.989936,0.736602,1.408328
...,...,...,...,...,...,...
995,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
996,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
997,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
998,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
# Calculate the mean (average) and standard deviation, rounded to 2 decimal places
avg_values = metrics_df.mean().round(2)
std_values = metrics_df.std(ddof=0).round(2)  # Use ddof=0 for population standard deviation

# Combine the average and standard deviation into the format "avg ± std"
combined_values = avg_values.astype(str) + " ± " + std_values.astype(str)

# Insert the LLM name at the beginning of the combined values
combined_values = combined_values.tolist()
combined_values.insert(0, LLM_name)

In [18]:
# Create a DataFrame for the combined average ± std row
result_df = pd.DataFrame([combined_values], columns=['Model'] + metrics_df.columns.tolist())
result_df

,Model,UniEval Naturalness,UniEval Coherence,UniEval Engagingness,UniEval Groundedness,UniEval Understandability,UniEval Overall
0,Qwen2-7B-Instruct,0.86 ± 0.24,0.94 ± 0.24,2.72 ± 1.44,0.68 ± 0.38,0.87 ± 0.24,1.21 ± 0.41


In [19]:
import pandas as pd

# Load the existing Excel file and update or append the average row
output_path = f'../Evaluations/{Dataset}{COT_}-{TOPIC_MODE}-results.xlsx'

try:
    # Load existing data
    existing_df = pd.read_excel(output_path)
    
    # Check if the model name already exists
    if LLM_name in existing_df['Model'].values:
        # Update the row by appending the new columns
        existing_index = existing_df.loc[existing_df['Model'] == LLM_name].index[0]
        for col in result_df.columns:
            if col not in existing_df.columns:
                existing_df[col] = None  # Add new column if missing
            existing_df.at[existing_index, col] = result_df[col].values[0]  # Update column values
    else:
        # Append the new data
        existing_df = pd.concat([existing_df, result_df], ignore_index=True)
except FileNotFoundError:
    # If the file does not exist, create a new DataFrame
    existing_df = result_df

# # Save the updated DataFrame to an Excel file
existing_df.to_excel(output_path, index=False)

existing_df


,Model,BLEU,R1,R2,RL,METEOR,BERTScore_Prec,BERTScore_Rec,BERTScore_F1,Dist1,...,UE Score,Persona Distance,response_time,Failure Ratio,UniEval Naturalness,UniEval Coherence,UniEval Engagingness,UniEval Groundedness,UniEval Understandability,UniEval Overall
0,Qwen2-7B-Instruct,0.02 ± 0.06,0.16 ± 0.11,0.04 ± 0.08,0.15 ± 0.11,0.17 ± 0.12,0.8 ± 0.2,0.81 ± 0.21,0.81 ± 0.2,0.83 ± 0.22,...,0.51 ± 0.79,0.29 ± 0.18,6.46 ± 2.92,0.06 ± 0.00,0.86 ± 0.24,0.94 ± 0.24,2.72 ± 1.44,0.68 ± 0.38,0.87 ± 0.24,1.21 ± 0.41


In [20]:
response = pd.read_excel(f'../Evaluations/{Dataset}{COT_}-{TOPIC_MODE}-results.xlsx')
response

,Model,BLEU,R1,R2,RL,METEOR,BERTScore_Prec,BERTScore_Rec,BERTScore_F1,Dist1,...,UE Score,Persona Distance,response_time,Failure Ratio,UniEval Naturalness,UniEval Coherence,UniEval Engagingness,UniEval Groundedness,UniEval Understandability,UniEval Overall
0,Qwen2-7B-Instruct,0.02 ± 0.06,0.16 ± 0.11,0.04 ± 0.08,0.15 ± 0.11,0.17 ± 0.12,0.8 ± 0.2,0.81 ± 0.21,0.81 ± 0.2,0.83 ± 0.22,...,0.51 ± 0.79,0.29 ± 0.18,6.46 ± 2.92,0.06 ± 0.00,0.86 ± 0.24,0.94 ± 0.24,2.72 ± 1.44,0.68 ± 0.38,0.87 ± 0.24,1.21 ± 0.41
